# Predict genomic coverage from DNA sequence

Because `momics` can ingest reference genome sequence as well as genomic coverage data, we can use it to predict genomic coverage from DNA sequence. This is useful for generating synthetic data or for filling in missing data in a dataset.

## Connect to the data repository

Here again, we will tap into the repository generated in the [previous tutorial](integrating-multiomics.ipynb). 

In [ ]:
from momics.momics import Momics

## Creating repository
repo = Momics("yeast_CNN_data.momics")

## Check that sequence and some tracks are registered
repo.seq()
repo.tracks()


## Define datasets and model 

We will define a simple convolutional neural network with `tensorflow` to predict the target variable `mnase` from the feature variable `seq` (the genome reference sequence). This requires to first define a set of genomic coordinates to extract genomic data from. We will extract sequences over tiling genomic windows (`features_size` of `8193`, with a stride of `48`) as feature variables to predict `mnase_rescaled` coverage scores over the same tiling genomic windows, but narrowed down to the a `target_size` of `128` bp around the center of the window. We can split the data into training, testing and validation sets, using `momics.utils.split_ranges()`.

In [ ]:
import momics.utils as mutils
from momics.dataset import MomicsDataset

# Fetch data from the momics repository
features = "nucleotide"
target = "atac_rescaled"
features_size = 8192 + 1
stride = 48
target_size = 512
batch_size = 500

bins = repo.bins(width=features_size, stride=stride, cut_last_bin_out=True)
bins = bins.subset(lambda x: x.Chromosome != "XVI")
bins_split, bins_test = mutils.split_ranges(bins, 0.8, shuffle=False)
bins_train, bins_val = mutils.split_ranges(bins_split, 0.8, shuffle=False)

train_dataset = (
    MomicsDataset(repo, bins_train, features, target, target_size=target_size, batch_size=batch_size).prefetch(20).repeat()
)
val_dataset = MomicsDataset(repo, bins_val, features, target, target_size=target_size, batch_size=batch_size).repeat()
test_dataset = MomicsDataset(repo, bins_test, features, target, target_size=target_size, batch_size=batch_size)


Now is time to define the model architecture. In this example, we will use a neural network adapted from `Basenji`, pre-defined in `momics.nn`. We can instantiate the model, and compile it with the desired optimizer, loss function and metrics.

In [ ]:
from momics import nn
from momics.nn import mae_cor
import tensorflow as tf  # type: ignore
from tensorflow.keras import layers  # type: ignore

model = nn.Basenji(input=layers.Input(shape=(features_size, 4)), output=layers.Dense(target_size, activation="linear")).model


def loss_fn(y_true, y_pred):
    return mae_cor(y_true, y_pred, alpha=0.9)


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=loss_fn,
    metrics=["mae"],
)
model.summary()


## Fit the model 

Now that we have the datasets and the model, we can fit the model to the training data, using the `fit()` method of the model. We can also evaluate the model on the testing and validation datasets.

In [ ]:
import numpy as np
from pathlib import Path
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau  # type: ignore

callbacks_list = [
    CSVLogger(Path(".chromnn", "seq2cov.basenji.epoch_data.csv")),
    ModelCheckpoint(
        filepath=Path(".chromnn_seq2cov", "Checkpoint.seq2cov.basenji.keras"), monitor="val_loss", save_best_only=True
    ),
    EarlyStopping(monitor="val_loss", patience=40, min_delta=1e-5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=6 // 2, min_lr=0.1 * 0.001),
]
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=callbacks_list,
    steps_per_epoch=int(np.floor(len(bins_train) // batch_size)),
    validation_steps=int(np.floor(len(bins_val) // batch_size)),
)


## Evaluate and save model 

Now let's see how the trained model performs, and save it to the local repository.

In [ ]:
# Evaluate the model
model.evaluate(test_dataset)


## Use the model to predict MNase-seq coverage

We can now use our trained model to predict ATAC-seq coverage from MNase-seq coverage, for example on a chromosome which has not been used for training.

In [ ]:
from momics import dataset as mmd
from momics import aggregate as mma

## Now predict the ATAC signal from the MNase signal
bb = repo.bins(width=features_size, stride=8, cut_last_bin_out=True)["XVI"]
ds = mmd.MomicsDataset(repo, bb, "nucleotide", batch_size=1000).prefetch(10)
predictions = model.predict(ds)

## Export predictions as a bigwig
bb3 = bb.copy()
bb3.Start = bb3.Start + features_size // 2 - target_size // 2
bb3.End = bb3.Start + target_size
chrom_sizes = {chrom: length for chrom, length in zip(repo.chroms().chrom, repo.chroms().length)}
keys = [f"{chrom}:{start}-{end}" for chrom, start, end in zip(bb3.Chromosome, bb3.Start, bb3.End)]
res = {f"atac-from-seq-basenji_f{features_size}_s{stride}_t{target_size}": {k: None for k in keys}}
for i, key in enumerate(keys):
    res[f"atac-from-seq-basenji_f{features_size}_s{stride}_t{target_size}"][key] = predictions[i]

mma.aggregate(res, bb3, chrom_sizes, type="mean", prefix="prediction")
